In [32]:
%pip install pandas wikipedia beautifulsoup4 lxml html5lib certifi requests


[notice] A new release of pip is available: 24.3.1 -> 26.0.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import requests
import certifi
import yfinance as yf
import io
# Prefer reading directly from the Wikipedia URL (may raise SSL errors on some setups)
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
# Try direct read_html from URL first
try:
    df_list = pd.read_html(url)
    df = df_list[0]
except Exception:
    # Try fetching the page using requests with certifi's CA bundle (secure)
    try:
        headers = {'User-Agent': 'python-requests/2.0'}
        resp = requests.get(url, headers=headers, timeout=15, verify=certifi.where())
        resp.raise_for_status()
        # Pass bytes to pandas so lxml doesn't treat the string as a filename
        df_list = pd.read_html(io.BytesIO(resp.content))
        df = df_list[0]
    except Exception:
        # Last resort: use wikipedia package HTML (wrap in StringIO for pandas)
        import wikipedia as wp
        html = wp.page('List of S&P 500 companies').html()
        df_list = pd.read_html(io.StringIO(html))
        df = df_list[0]
# Now select only company name and ticker columns (robust to column name variations)
cols_low = [str(c).lower() for c in df.columns.astype(str)]
company_candidates = ['security','company','company name','name','firm']
ticker_candidates = ['symbol','ticker','ticker symbol']
company_col = None
ticker_col = None
for orig, low in zip(df.columns, cols_low):
    if any(k in low for k in company_candidates) and company_col is None:
        company_col = orig
    if any(k in low for k in ticker_candidates) and ticker_col is None:
        ticker_col = orig
# Fallback heuristics
if company_col is None:
    for orig in df.columns:
        if df[orig].dtype == object:
            company_col = orig
            break
if ticker_col is None:
    for orig in df.columns:
        non_null = df[orig].dropna().astype(str)
        if len(non_null) == 0:
            continue
        sample = non_null.iloc[0]
        if 1 <= len(sample) <= 6 and sample.upper() == sample:
            ticker_col = orig
            break
if ticker_col is None:
    ticker_col = df.columns[1] if len(df.columns) > 1 else df.columns[0]
out = df[[company_col, ticker_col]].copy()
out.columns = ['Company', 'Ticker']
out['Company'] = out['Company'].astype(str).str.replace(r'\[.*?\]', '', regex=True).str.strip()
out['Ticker'] = out['Ticker'].astype(str).str.strip()

ticker_list = out['Ticker'].tolist()
raw_data = yf.download(ticker_list, start="2023-01-01", end="2025-01-01", group_by='column')
print(raw_data['Close'])

[****************      33%                       ]  167 of 503 completed$BF.B: possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01)
[********************* 43%                       ]  214 of 503 completed$Q: possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01) (Yahoo error = "Data doesn't exist for startDate = 1672549200, endDate = 1735707600")
[**********************57%**                     ]  286 of 503 completed$SNDK: possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01) (Yahoo error = "Data doesn't exist for startDate = 1672549200, endDate = 1735707600")
[**********************80%*************          ]  400 of 503 completed$BRK.B: possibly delisted; no timezone found
[*********************100%***********************]  503 of 503 completed

4 Failed downloads:
['BF.B']: possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01)
['Q', 'SNDK']: possibly delisted; no price data found  (1d 2023-01-01 -> 2025-01-01) (Yahoo e

Ticker               A        AAPL        ABBV        ABNB         ABT  \
Date                                                                     
2023-01-03  146.749908  123.096024  144.322632   84.900002  102.849228   
2023-01-04  148.344177  124.365662  145.486954   88.720001  104.379097   
2023-01-05  148.774506  123.046806  145.309174   87.709999  103.994293   
2023-01-06  144.431854  127.574203  148.028915   88.519997  105.430321   
2023-01-09  144.236252  128.095856  143.682739   89.239998  105.261368   
...                ...         ...         ...         ...         ...   
2024-12-24  134.764069  256.797180  172.530685  134.990005  112.058289   
2024-12-26  134.496735  257.612701  171.763855  135.320007  112.556282   
2024-12-27  134.209534  254.201370  170.623260  133.384995  112.282875   
2024-12-30  133.100540  250.829773  168.888367  131.809998  110.144440   
2024-12-31  133.267197  249.059464  170.326111  131.410004  110.447128   

Ticker           ACGL         ACN    